In [1]:
import numpy as np
import pandas as pd
import pickle

# 1.Data
- Load data from pickle file 
- split the data into independent and dependent 
- split to train and test set

In [2]:
data = pickle.load(open('data_face_features.pickle',mode='rb'))

In [4]:
x = np.array(data['data']) # independent vriable
y = np.array(data ['label']) # dependent variable

In [6]:
x.shape, y.shape

((340, 1, 128), (340,))

In [7]:
x = x.reshape(-1,128)
x.shape

(340, 128)

In [9]:
# split the data into train and test
from sklearn.model_selection import train_test_split

In [10]:
x_train,x_test,y_train,y_test = train_test_split(x,y,train_size=0.8,random_state=0)

In [13]:
x_train.shape, x_test.shape, y_train.shape,y_test.shape

((272, 128), (68, 128), (272,), (68,))

# 2. Train Machine learning

In [14]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import VotingClassifier
from sklearn.metrics import classification_report,accuracy_score,f1_score

### Logistic Regression 

In [16]:
model_logistic = LogisticRegression()
model_logistic.fit(x_train,y_train) # training Logistic regression

LogisticRegression()

In [21]:
def get_report(model, x_train,y_train,x_test,y_test):  
    y_pred_train = model.predict(x_train)
    y_pred_test = model.predict(x_test)

    # accuracy score 
    acc_train = accuracy_score(y_train,y_pred_train)
    acc_test = accuracy_score(y_test,y_pred_test)

    # f1 score 
    f1_score_train = f1_score(y_train,y_pred_train,average='macro')
    f1_score_test= f1_score(y_test,y_pred_test,average='macro')


    print('Accuracy Train = %0.2f'%acc_train)
    print('Accuracy Test = %0.2f'%acc_test)
    print('F1 score Train = %0.2f'%f1_score_train)
    print('F1 score Test = %0.2f'%f1_score_test)

In [22]:
get_report(model_logistic,x_train,y_train,x_test,y_test)

Accuracy Train = 0.88
Accuracy Test = 0.91
F1 score Train = 0.88
F1 score Test = 0.91


# support vector machines 

In [23]:
model_svc = SVC(probability=True)
model_svc.fit(x_train,y_train)

SVC(probability=True)

In [24]:
get_report(model_svc,x_train,y_train,x_test,y_test)

Accuracy Train = 0.94
Accuracy Test = 0.91
F1 score Train = 0.94
F1 score Test = 0.91


# Random Forest

In [27]:
model_rf = RandomForestClassifier(n_estimators=10)
model_rf.fit(x_train,y_train)

RandomForestClassifier(n_estimators=10)

In [28]:
get_report(model_rf,x_train,y_train,x_test,y_test)

Accuracy Train = 1.00
Accuracy Test = 0.90
F1 score Train = 1.00
F1 score Test = 0.89


# Voting Classifier

In [30]:
model_voting= VotingClassifier(estimators=[
    ('logistic',LogisticRegression()),
    ('svm',SVC(probability=True)),
    ('rf',RandomForestClassifier())
], voting='soft',weights=[2,3,1])

In [31]:
model_voting.fit(x_train,y_train)

VotingClassifier(estimators=[('logistic', LogisticRegression()),
                             ('svm', SVC(probability=True)),
                             ('rf', RandomForestClassifier())],
                 voting='soft')

In [32]:
get_report(model_voting,x_train,y_train,x_test,y_test)

Accuracy Train = 0.96
Accuracy Test = 0.93
F1 score Train = 0.96
F1 score Test = 0.92


# Parameter Tuning

In [34]:
from sklearn.model_selection import GridSearchCV

In [41]:
model_grid = GridSearchCV(model_voting,
                         param_grid={
                             'svm__C':[3,5,7,10],
                             'svm__gamma':[0.1,0.3,0.5],
                             'rf__n_estimators':[5,10,20],
                             'rf__max_depth':[3,5,7],
                             'voting':['soft','hard']
                         },scoring='accuracy',cv=3,n_jobs=1,verbose=2)

In [42]:
model_grid.fit(x_train,y_train)

Fitting 3 folds for each of 216 candidates, totalling 648 fits
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.1, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=5, svm__C=3, svm__gamma=0.3, voting=soft; total time=   0.0s
[

[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.3, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=soft; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=3, svm__gamma=0.5, voting=hard; total time=   0.0s
[CV] END rf__max_depth=3, rf__n_estimators=10, svm__C=5

limit_output extension: Maximum message size of 10000 exceeded with 10089 characters

In [44]:
model_best_estimator = model_grid.best_estimator_

In [45]:
model_grid.best_score_

0.8750915750915751

# 4. save model

In [46]:
pickle.dump(model_best_estimator,open('./models/machinelearning_face_person_identity.pkl',mode='wb'))